```text
Non Linear Network
```

In [ ]:
# We import pytorch and torchvision
import torchvision, torch
import torch.nn as nn
import torch.nn.functional as F
# Import numpy
import numpy as np
import time

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,), (0.5,))  # Perform normalization on GPU if possible
])

# We load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='data/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='data/', train=False, transform=transform, download=True)

# Charger un dataloader with batch size x
def get_data_loader(dataset, batch_size):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

def get_batch_format(data_loader):
    images_t, labels_t = next(iter(data_loader))
    print('images.shape:', images_t.shape)
    print('labels.shape:', labels_t.shape)

class Network(nn.Module):
    def __init__(self, input_dim, hidden_dim1 = 32, output_dim=10):
        super(Network, self).__init__()
        self.a = nn.Linear(input_dim, hidden_dim1)
        self.b = nn.Linear(hidden_dim1, hidden_dim1)
        self.c = nn.Linear(hidden_dim1, hidden_dim1)
        self.d = nn.Linear(hidden_dim1, output_dim)
        
        # We Add non-linearity
        self.act = nn.ReLU()

    def forward(self, x):
        x = x.permute(0, 2, 3, 1)
        x1 = self.act(self.a(x))
        x2 = self.act(self.b(x1))
        x3 = self.act(self.c(x2))
        x4 = self.d(x3)
        return x4

# Fonction accuracy pour calculer le total des bonnes réponses
def accuracy(predictions, labels):
    _, predicted_labels = torch.max(predictions, 1)
    correct = (predicted_labels == labels).sum().item()
    return correct / len(labels)

# Charger dans un DataLoader le data set de train CIFAR10
train_loader = get_data_loader(train_dataset, 128) # On charge par batch
# On charge le dataset de testto
test_loader = get_data_loader(test_dataset, 10000) # On charge tout le dataset de test
# Print the format of the batch
get_batch_format(train_loader)

# On recupère le device
deviceGPU = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# On crée une instance de la classe MyClass
modelNet = Network(3*32*32, 32, 10)

# On envoie le modèle sur le device
modelNet = modelNet.to(deviceGPU)

# On crée une fonction de loss
lossFn = F.cross_entropy
learningRate = 0.0001

# On crée un optimiseur
opt = torch.optim.Adam(modelNet.parameters(), lr=learningRate, weight_decay=0.0001)

def off_load_on_gpu(train_loader, test_loader, device):
    train_loader_gpu = [(x.to(device).view(x.shape[0], -1), y.to(device)) for x, y in train_loader]
    test_loader_gpu = [(x.to(device).view(x.shape[0], -1), y.to(device)) for x, y in test_loader]
    return train_loader_gpu, test_loader_gpu

def training_cycle(model, train_loader, device):
    model.train()
    for x, y in train_loader:
        preds = model(x)
        loss = lossFn(preds, y)
        # Compute gradients
        loss.backward()
        # Update parameters
        opt.step()
        # Reset gradients to 0
        opt.zero_grad()
    
def validation_cycle(model, test_loader, epoch, num_epochs, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            preds = model(x)
            loss = lossFn(preds, y)
            total_loss += loss.item()
            total_correct += accuracy(preds, y)

    avg_loss = total_loss / len(test_loader)
    avg_acc = total_correct / len(test_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {avg_acc * 100:.2f}%")

def fit_one_cycle(model, train_loader, test_loader, epoch, num_epochs, device): 
    # Training
    training_cycle(model, train_loader, device)
    
    # Validation
    validation_cycle(model, test_loader, epoch, num_epochs, device)
    
# Training loop
num_epochs = 50
# Time to train
time_to_train = time.time()

# On charge les données sur le GPU
train_loader_gpu, test_loader_gpu = off_load_on_gpu(train_loader, test_loader, deviceGPU)

for epoch in range(num_epochs):
    fit_one_cycle(modelNet, train_loader_gpu, test_loader_gpu, epoch, num_epochs, deviceGPU)

time_to_train = time.time() - time_to_train
print(f"Time to train: {time_to_train:.2f} seconds")

Files already downloaded and verified
Files already downloaded and verified
images.shape: torch.Size([128, 3, 32, 32])
labels.shape: torch.Size([128])
Epoch 1/250, Loss: 1.8700, Accuracy: 33.28%
Epoch 2/250, Loss: 1.7406, Accuracy: 38.14%
Epoch 3/250, Loss: 1.6804, Accuracy: 39.97%
Epoch 4/250, Loss: 1.6440, Accuracy: 41.63%
Epoch 5/250, Loss: 1.6170, Accuracy: 42.68%
Epoch 6/250, Loss: 1.5952, Accuracy: 43.85%
Epoch 7/250, Loss: 1.5772, Accuracy: 44.33%
Epoch 8/250, Loss: 1.5619, Accuracy: 44.86%
Epoch 9/250, Loss: 1.5491, Accuracy: 45.30%
Epoch 10/250, Loss: 1.5383, Accuracy: 45.54%
Epoch 11/250, Loss: 1.5288, Accuracy: 46.06%
Epoch 12/250, Loss: 1.5197, Accuracy: 46.42%
Epoch 13/250, Loss: 1.5122, Accuracy: 46.73%
Epoch 14/250, Loss: 1.5055, Accuracy: 46.97%
Epoch 15/250, Loss: 1.4994, Accuracy: 47.22%
Epoch 16/250, Loss: 1.4944, Accuracy: 47.30%
Epoch 17/250, Loss: 1.4892, Accuracy: 47.47%
Epoch 18/250, Loss: 1.4850, Accuracy: 47.72%
Epoch 19/250, Loss: 1.4809, Accuracy: 47.84%
Epo

```text
Convolutionnal Network
```

In [13]:
# We import pytorch and torchvision
import torchvision, torch
import torch.nn as nn
import torch.nn.functional as F
# Import numpy
import numpy as np
import time

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,), (0.5,))  # Perform normalization on GPU if possible
])

# We load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='data/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='data/', train=False, transform=transform, download=True)

# Charger un dataloader with batch size x
def get_data_loader(dataset, batch_size):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

def get_batch_format(data_loader):
    images_t, labels_t = next(iter(data_loader))
    print('images.shape:', images_t.shape)
    print('labels.shape:', labels_t.shape)

# Fonction accuracy pour calculer le total des bonnes réponses
def accuracy(predictions, labels):
    _, predicted_labels = torch.max(predictions, 1)
    correct = (predicted_labels == labels).sum().item()
    return correct / len(labels)

# Charger dans un DataLoader le data set de train CIFAR10
train_loader = get_data_loader(train_dataset, 128) # On charge par batch
# On charge le dataset de testto
test_loader = get_data_loader(test_dataset, 10000) # On charge tout le dataset de test
# Print the format of the batch
get_batch_format(train_loader)

# On recupère le device
deviceGPU = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# On crée une fonction de loss
lossFn = F.cross_entropy
learningRate = 0.0001

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.act(self.conv1(x)))
        x = self.pool(self.act(self.conv2(x)))
        x = self.pool(self.act(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = self.act(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the ConvNet model
modelNet = ConvNet().to(deviceGPU)

# Create a new optimizer for the ConvNet model
opt = torch.optim.Adam(modelNet.parameters(), lr=learningRate, weight_decay=0.0001)

def off_load_on_gpu(train_loader, test_loader, device):
    train_loader_gpu = [(x.to(device), y.to(device)) for x, y in train_loader]
    test_loader_gpu = [(x.to(device), y.to(device)) for x, y in test_loader]
    return train_loader_gpu, test_loader_gpu

def training_cycle(model, train_loader, device):
    model.train()
    for x, y in train_loader:
        preds = model(x)
        loss = lossFn(preds, y)
        # Compute gradients
        loss.backward()
        # Update parameters
        opt.step()
        # Reset gradients to 0
        opt.zero_grad()
    
def validation_cycle(model, test_loader, epoch, num_epochs, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            preds = model(x)
            loss = lossFn(preds, y)
            total_loss += loss.item()
            total_correct += accuracy(preds, y)

    avg_loss = total_loss / len(test_loader)
    avg_acc = total_correct / len(test_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {avg_acc * 100:.2f}%")

def fit_one_cycle(model, train_loader, test_loader, epoch, num_epochs, device): 
    # Training
    training_cycle(model, train_loader, device)
    
    # Validation
    validation_cycle(model, test_loader, epoch, num_epochs, device)

# Offload data to GPU
train_loader_gpu, test_loader_gpu = off_load_on_gpu(train_loader, test_loader, deviceGPU)

# Training loop
num_epochs = 50
time_to_train = time.time()

for epoch in range(num_epochs):
    fit_one_cycle(modelNet, train_loader_gpu, test_loader_gpu, epoch, num_epochs, deviceGPU)

time_to_train = time.time() - time_to_train
print(f"Time to train: {time_to_train:.2f} seconds")

Files already downloaded and verified
Files already downloaded and verified
images.shape: torch.Size([128, 3, 32, 32])
labels.shape: torch.Size([128])
Epoch 1/50, Loss: 1.5867, Accuracy: 42.42%
Epoch 2/50, Loss: 1.4441, Accuracy: 47.74%
Epoch 3/50, Loss: 1.3621, Accuracy: 51.12%
Epoch 4/50, Loss: 1.3019, Accuracy: 53.50%
Epoch 5/50, Loss: 1.2501, Accuracy: 55.31%
Epoch 6/50, Loss: 1.2048, Accuracy: 57.11%
Epoch 7/50, Loss: 1.1648, Accuracy: 58.82%
Epoch 8/50, Loss: 1.1294, Accuracy: 59.90%
Epoch 9/50, Loss: 1.0989, Accuracy: 60.91%
Epoch 10/50, Loss: 1.0711, Accuracy: 61.99%
Epoch 11/50, Loss: 1.0452, Accuracy: 63.05%
Epoch 12/50, Loss: 1.0228, Accuracy: 63.93%
Epoch 13/50, Loss: 1.0036, Accuracy: 64.66%
Epoch 14/50, Loss: 0.9865, Accuracy: 65.33%
Epoch 15/50, Loss: 0.9702, Accuracy: 65.80%
Epoch 16/50, Loss: 0.9560, Accuracy: 66.60%
Epoch 17/50, Loss: 0.9424, Accuracy: 67.02%
Epoch 18/50, Loss: 0.9302, Accuracy: 67.58%
Epoch 19/50, Loss: 0.9190, Accuracy: 67.93%
Epoch 20/50, Loss: 0.9

```text
Strided Convolutions
```

In [ ]:
# We import pytorch and torchvision
import torchvision, torch
import torch.nn as nn
import torch.nn.functional as F
# Import numpy
import numpy as np
import time

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,), (0.5,))  # Perform normalization on GPU if possible
])

# We load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='data/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='data/', train=False, transform=transform, download=True)

# Charger un dataloader with batch size x
def get_data_loader(dataset, batch_size):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

def get_batch_format(data_loader):
    images_t, labels_t = next(iter(data_loader))
    print('images.shape:', images_t.shape)
    print('labels.shape:', labels_t.shape)

# Fonction accuracy pour calculer le total des bonnes réponses
def accuracy(predictions, labels):
    _, predicted_labels = torch.max(predictions, 1)
    correct = (predicted_labels == labels).sum().item()
    return correct / len(labels)

# Charger dans un DataLoader le data set de train CIFAR10
train_loader = get_data_loader(train_dataset, 128) # On charge par batch
# On charge le dataset de testto
test_loader = get_data_loader(test_dataset, 10000) # On charge tout le dataset de test
# Print the format of the batch
get_batch_format(train_loader)

# On recupère le device
deviceGPU = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# On crée une fonction de loss
lossFn = F.cross_entropy
learningRate = 0.0001
  
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)  # Stride 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)  # Stride 2
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)  # Stride 2
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.act(self.conv1(x))
        x = self.act(self.conv2(x))
        x = self.act(self.conv3(x))
        x = x.view(-1, 128 * 4 * 4)
        x = self.act(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the ConvNet model
modelNet = ConvNet().to(deviceGPU)

# Create a new optimizer for the ConvNet model
opt = torch.optim.Adam(modelNet.parameters(), lr=learningRate, weight_decay=0.0001)

def off_load_on_gpu(train_loader, test_loader, device):
    train_loader_gpu = [(x.to(device), y.to(device)) for x, y in train_loader]
    test_loader_gpu = [(x.to(device), y.to(device)) for x, y in test_loader]
    return train_loader_gpu, test_loader_gpu

def training_cycle(model, train_loader, device):
    model.train()
    for x, y in train_loader:
        preds = model(x)
        loss = lossFn(preds, y)
        # Compute gradients
        loss.backward()
        # Update parameters
        opt.step()
        # Reset gradients to 0
        opt.zero_grad()
    
def validation_cycle(model, test_loader, epoch, num_epochs, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            preds = model(x)
            loss = lossFn(preds, y)
            total_loss += loss.item()
            total_correct += accuracy(preds, y)

    avg_loss = total_loss / len(test_loader)
    avg_acc = total_correct / len(test_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {avg_acc * 100:.2f}%")

def fit_one_cycle(model, train_loader, test_loader, epoch, num_epochs, device): 
    # Training
    training_cycle(model, train_loader, device)
    
    # Validation
    validation_cycle(model, test_loader, epoch, num_epochs, device)

# Offload data to GPU
train_loader_gpu, test_loader_gpu = off_load_on_gpu(train_loader, test_loader, deviceGPU)

# Training loop
num_epochs = 50
time_to_train = time.time()

for epoch in range(num_epochs):
    fit_one_cycle(modelNet, train_loader_gpu, test_loader_gpu, epoch, num_epochs, deviceGPU)

time_to_train = time.time() - time_to_train
print(f"Time to train: {time_to_train:.2f} seconds")

Files already downloaded and verified
Files already downloaded and verified
images.shape: torch.Size([128, 3, 32, 32])
labels.shape: torch.Size([128])
